In [15]:
import pickle
import numpy as np

from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# ---------------------------------------------------
# 0. Load dataset once
# ---------------------------------------------------

with open("dataset_description.pkl", "rb") as f:
    dc = pickle.load(f)

X = dc["latent_vars"]        # shape (N, d)
attrs = dc["attributes"]     # shape (N, 73)
fields = dc["fields"]        # list of 73 attribute names

print("Loaded dataset_description.pkl")
print("X shape:", X.shape)
print("attrs shape:", attrs.shape)
print("Number of attributes:", len(fields))

# Target attribute is fixed (Smiling)
target_name = "Frowning"
j_y = fields.index(target_name)
y = (attrs[:, j_y] > 0).astype(int)

print(f"\nFixed target attribute = {target_name}")
print("Positive rate (y=1): {:.3f}".format(y.mean()))

# ---------------------------------------------------
# Helper functions (metrics + non-DP multiaccuracy)
# ---------------------------------------------------

def report_metrics(name, y_true, p, g=None):
    """
    Print overall accuracy and (optionally) per-group accuracies.

    Args:
        name: String name for the model
        y_true: True labels (0/1), shape (n,)
        p: Predicted probabilities in [0,1], shape (n,)
        g: Protected group labels (0/1), shape (n,) or None
    """
    y_pred = (p >= 0.5).astype(int)
    acc = accuracy_score(y_true, y_pred)
    print(f"=== {name} ===")
    print(f"Accuracy: {acc:.3f}")

    if g is not None:
        accs = {}
        for val in [0, 1]:
            mask = (g == val)
            if mask.sum() == 0:
                continue
            acc_g = accuracy_score(y_true[mask], y_pred[mask])
            accs[val] = acc_g
            print(f"  Group {val} accuracy ({mask.sum()} samples): {acc_g:.3f}")
        if 0 in accs and 1 in accs:
            gap = abs(accs[0] - accs[1])
            print(f"  |accuracy gap| (group 0 vs 1): {gap:.3f}")
    print()


def multiaccuracy_update(probs, h_vals, eta):
    """
    Multiaccuracy multiplicative-weights-style update on probabilities.

    We interpret probs as sigmoid(logits) and move logits
    in direction -eta * h(x), then map back through sigmoid.

    Args:
        probs: Current probabilities in [0,1], shape (n,)
        h_vals: Auditor outputs h_t(x), shape (n,)
        eta: Learning rate for the update

    Returns:
        Updated probabilities in [0,1], shape (n,)
    """
    # Avoid numerical issues at 0 and 1
    eps = 1e-6
    p = np.clip(probs, eps, 1 - eps)

    # logit(p) = log(p / (1-p))
    logits = np.log(p / (1.0 - p))

    # Update logits by moving in the opposite direction of auditor prediction
    new_logits = logits - eta * h_vals

    # Map back through sigmoid
    new_probs = 1.0 / (1.0 + np.exp(-new_logits))
    return new_probs


def run_non_dp_multiaccuracy_boost(
    X_audit, y_audit,
    X_test, y_test,
    base_model,
    T=5,
    eta=0.1,
    stop_threshold=0.005,
    seed=0,
):
    """
    Non-DP multiaccuracy boosting (no privacy, exact correlation).

    We:
      1) Start from base_model probabilities on audit and test.
      2) At each round t:
         - Fit a linear regression auditor to residuals on the audit set.
         - Compute exact correlation delta_t = E[h_t(x)*(f_t(x)-y)] on audit.
         - If |delta_t| < stop_threshold, stop.
         - Otherwise update f_t on both audit and test using multiaccuracy_update.

    Args:
        X_audit, y_audit: audit set
        X_test, y_test: test set (for evaluation of post-processed classifier)
        base_model: already-fitted probabilistic classifier with predict_proba
        T: maximum number of boosting rounds
        eta: multiaccuracy learning rate
        stop_threshold: stop if |delta_t| < this value
        seed: random seed for reproducibility

    Returns:
        dict with:
            "fT_audit": final probabilities on audit
            "fT_test" : final probabilities on test
            "history" : dict with "delta" (exact correlations per round)
    """
    rng = np.random.RandomState(seed)

    # Start from base-model probabilities
    f_audit = base_model.predict_proba(X_audit)[:, 1]
    f_test = base_model.predict_proba(X_test)[:, 1]

    deltas = []

    for t in range(T):
        # Compute residuals on audit
        residuals = f_audit - y_audit  # shape (n_audit,)

        # Fit a linear regression auditor h_t(x) ≈ residual
        auditor = LinearRegression()
        auditor.fit(X_audit, residuals)

        # Auditor predictions on audit and test
        h_audit = auditor.predict(X_audit)
        h_test = auditor.predict(X_test)

        # Exact correlation on the audit set
        delta_t = np.mean(h_audit * (f_audit - y_audit))
        deltas.append(delta_t)

        print(f"[Non-DP MA] Round {t}: correlation delta_t = {delta_t:.6f}")

        # Stopping condition
        if abs(delta_t) < stop_threshold:
            print(f"Stopping early at round {t} (|delta_t| < {stop_threshold}).")
            break

        # Multiaccuracy update on both audit and test probabilities
        f_audit = multiaccuracy_update(f_audit, h_audit, eta)
        f_test = multiaccuracy_update(f_test, h_test, eta)

    return {
        "fT_audit": f_audit,
        "fT_test": f_test,
        "history": {
            "delta": deltas,
        },
    }


# ---------------------------------------------------
# 1. Import DP multiaccuracy routine
# ---------------------------------------------------
from dp_multiaccuracy_utils import run_dp_multiaccuracy_boost

# Hyperparameters you can tune
T_ma_non_dp = 10          # rounds for non-DP MA
T_ma_dp = 10             # rounds for DP MA
eta_non_dp = 0.1
eta_dp = 0.1

epsilon_round = 1      # per-round epsilon (DP)
delta_round   = 1e-2     # per-round delta (DP)

clipping_grad = 1.0
clipping_corr = 1.0
auditor_lr = 0.05
auditor_steps = 200
auditor_batch_size = 256
stop_threshold_dp = 0.002


# ---------------------------------------------------
# 2. Loop over protected attributes: Indian, Asian, White, Black
# ---------------------------------------------------

protected_list = ["Indian", "Asian", "White", "Black"]

for protected_name in protected_list:
    print("\n" + "="*80)
    print(f"Protected attribute = {protected_name}")

    # Construct protected group label g for this attribute
    j_g = fields.index(protected_name)
    g = (attrs[:, j_g] > 0).astype(int)

    print("Group 1 fraction (g=1): {:.3f}".format(g.mean()))

    # Train / Audit / Test split (stratify only on y so splits are comparable)
    X_train, X_tmp, y_train, y_tmp, g_train, g_tmp = train_test_split(
        X, y, g, test_size=0.4, random_state=0, stratify=y
    )

    X_audit, X_test, y_audit, y_test, g_audit, g_test = train_test_split(
        X_tmp, y_tmp, g_tmp, test_size=0.5, random_state=1, stratify=y_tmp
    )

    print("\nDataset sizes:")
    print("  Train: ", X_train.shape[0])
    print("  Audit: ", X_audit.shape[0])
    print("  Test  : ", X_test.shape[0])

    # ------------------------------
    # Base model (no MA, no DP)
    # ------------------------------
    base = LogisticRegression(max_iter=1000, solver="lbfgs")
    base.fit(X_train, y_train)

    p_test_base = base.predict_proba(X_test)[:, 1]
    report_metrics("Base model", y_test, p_test_base, g_test)

    # ------------------------------
    # Non-DP multiaccuracy
    # ------------------------------
    results_ma = run_non_dp_multiaccuracy_boost(
        X_audit, y_audit,
        X_test, y_test,
        base_model=base,
        T=T_ma_non_dp,
        eta=eta_non_dp,
        stop_threshold=0.005,
        seed=0,
    )

    p_test_ma = results_ma["fT_test"]
    report_metrics("Non-DP multiaccuracy (post-processed)", y_test, p_test_ma, g_test)
    print("Exact correlations per round (non-DP):", results_ma["history"]["delta"])
    print()

    # ------------------------------
    # DP multiaccuracy
    # ------------------------------
    results_dp = run_dp_multiaccuracy_boost(
        X_train, y_train,
        X_audit, y_audit,
        X_test, y_test,
        base_model=base,
        T=T_ma_dp,
        eta=eta_dp,
        epsilon_round=epsilon_round,
        delta_round=delta_round,
        clipping_grad=clipping_grad,
        clipping_corr=clipping_corr,
        auditor_lr=auditor_lr,
        auditor_steps=auditor_steps,
        auditor_batch_size=auditor_batch_size,
        stop_threshold=0.005,
        seed=0,
    )

    p_test_dp = results_dp["fT_test"]
    report_metrics("DP multiaccuracy (post-processed)", y_test, p_test_dp, g_test)
    print("Noisy correlation estimates per round (DP):", results_dp["history"]["delta_hat"])
    print()


Loaded dataset_description.pkl
X shape: (13143, 100)
attrs shape: (13143, 73)
Number of attributes: 73

Fixed target attribute = Frowning
Positive rate (y=1): 0.581

Protected attribute = Indian
Group 1 fraction (g=1): 0.023

Dataset sizes:
  Train:  7885
  Audit:  2629
  Test  :  2629
=== Base model ===
Accuracy: 0.713
  Group 0 accuracy (2573 samples): 0.714
  Group 1 accuracy (56 samples): 0.643
  |accuracy gap| (group 0 vs 1): 0.071

[Non-DP MA] Round 0: correlation delta_t = 0.010194
[Non-DP MA] Round 1: correlation delta_t = 0.009811
[Non-DP MA] Round 2: correlation delta_t = 0.009442
[Non-DP MA] Round 3: correlation delta_t = 0.009087
[Non-DP MA] Round 4: correlation delta_t = 0.008745
[Non-DP MA] Round 5: correlation delta_t = 0.008416
[Non-DP MA] Round 6: correlation delta_t = 0.008099
[Non-DP MA] Round 7: correlation delta_t = 0.007795
[Non-DP MA] Round 8: correlation delta_t = 0.007501
[Non-DP MA] Round 9: correlation delta_t = 0.007219
=== Non-DP multiaccuracy (post-process